In [2]:
# 准备数据集
from datasets import load_dataset

ds = load_dataset("doushabao4766/msra_ner_k_V3")
# 数据清洗,，处理空数据
def filter_data(example):
    return len(example['tokens']) > 0 and len(example['ner_tags']) > 0
for split in ds.keys():
    ds[split] = ds[split].filter(filter_data)


Filter: 100%|██████████| 3443/3443 [00:00<00:00, 34644.46 examples/s]


In [4]:
# 保存数据集
import pickle
with open('./ds.bin','wb') as f:
    pickle.dump(ds,f)

In [12]:
# 加载数据集
import pickle
with open('./ds.bin','rb') as f:
    ds = pickle.load(f)
# 数据集
j = 0
for i in ds['train']:
    print(i['tokens'])
    print(i['ner_tags'])
    print(i['knowledge'])
    j += 1
    if j > 1:
        break
ds


['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

['藏', '书', '本', '来', '就', '是', '所', '有', '传', '统', '收', '藏', '门', '类', '中', '的', '第', '一', '大', '户', '，', '只', '是', '我', '们', '结', '束', '温', '饱', '的', '时', '间', '太', '短', '而', '已', '。']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
收藏：收藏是一种对于物品的搜集、储存、分类与维护的癖好。收藏家的收集对象通常是有价值的古董，但也可能是其他的小东西。集邮、火柴盒贴画与明信片是较为主流的收集项目。


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 3442
    })
})

In [13]:
# 实体映射数据集词典准备
entities = ['O'] + list({'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC',
           'I-LOC'})
entities_index = {entity:i for i,entity in enumerate(entities)}
print(entities_index)

{'O': 0, 'I-ORG': 1, 'B-ORG': 2, 'I-PER': 3, 'B-LOC': 4, 'B-PER': 5, 'I-LOC': 6}


In [14]:
# 模型准备
from transformers import AutoModelForTokenClassification,AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', num_labels=7)
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# tokenizer处理
def text2token(example):
    result = tokenizer(example['tokens'],padding=True,add_special_tokens=False,truncation=True,max_length=512,is_split_into_words=True)
    word2ids = result.word_ids()
    adjust_labels = []
    i,prev_id = -1,-1
    for wid in word2ids:
        if wid != prev_id:
            i += 1
            prev_id = wid
        adjust_labels.append(example['ner_tags'][i])
    result['labels'] = adjust_labels
    return result
ds_token = ds.map(text2token)


Map: 100%|██████████| 45000/45000 [00:09<00:00, 4518.85 examples/s]


In [17]:
ds_token

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3442
    })
})

In [18]:
print(ds_token["train"][0].keys())
print(ds_token["train"][0]["tokens"])    
print(ds_token["train"][0]["ner_tags"])
print(ds_token['train'][0]['labels'])

dict_keys(['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'])
['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## 模型训练

In [19]:
from transformers import TrainingArguments, Trainer
# TrainingArguments
training_args = TrainingArguments(
    output_dir='ner_train',
    num_train_epochs=3,
    save_safetensors=False,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    report_to='tensorboard',
    eval_strategy='epoch'
)

In [20]:
id2lbl = {v:k for k,v in entities_index.items()}
lbl2id = entities_index

In [21]:
# 初始化模型
model = AutoModelForTokenClassification.from_pretrained(
    'google-bert/bert-base-chinese',
    num_labels=7,
    id2label = id2lbl,
    label2id = lbl2id
)
model

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [22]:
import evaluate
import numpy as np
# Trainer
def compute_metric(result):
    seqeavl = evaluate.load('seqeval')
    predicts,labels = result
    predicts = np.argmax(predicts,axis=2)

    # 准备评估数据
    predicts = [[entities[p] for p,l in zip(ps,ls) if l!=-100]
                for ps,ls in zip(predicts,labels)]
    labels = [[entities[l] for p,l in zip(ps,ls) if l!=-100]
                for ps,ls in zip(predicts,labels)]
    results = seqeavl.compute(predictions=predicts,references=labels)
    return results

In [24]:
# 解决长度不一致的问题，将长度不一致的标签列表用-100进行填充
from transformers import DataCollatorForTokenClassification 
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer,padding=True)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_token['train'],
    eval_dataset=ds_token['test'],
    data_collator=data_collator,
    compute_metrics=compute_metric
)

In [ ]:
# 开始训练
trainer.train()

In [ ]:
# 模型推理
result = trainer.predict(ds_token['test'])

In [ ]:
print(ds_token['test'][10]['tokens'])
print(ds_token['test'][10]['ner_tags'])
print(result.label_ids[10])

In [ ]:
[entities[p] for p,l in zip(result.label_ids[10],ds_token['test'][10]['ner_tags'])]

In [ ]:
[entities[l] for p,l in zip(result.label_ids[10],ds_token['test'][10]['ner_tags'])]